## 1. Data retrieval

In [24]:
import torch
import gzip
import json
import pandas as pd

from operator import itemgetter
from summarizer import Summarizer

### Data directory and batch size selection

In [29]:
# Modify this to wherever you locally downloaded the data
data_base_path = './data/newsroom-release/release/'
wordpiece_cased_path = 'bert-base-cased-vocab.txt'

# train_path = data_base_path + 'train.jsonl.gz' DONT USE THIS
validation_path = data_base_path + 'dev.jsonl.gz'
test_path = data_base_path + 'dev.jsonl.gz'

batch_size = 1

In [30]:
class NewsroomDataset(torch.utils.data.Dataset):
    '''
    Attributes:
        batch_size: Batch size to be taken on single getitem
        file: path to the dataset file
        category: category of the data summarization. i.e. 'extractive'
    '''
    def __init__(self, path, category: str):
        self.category = category
        data = []
        with gzip.open(path) as f:
            for ln in f:
                obj = json.loads(ln)
                data.append(obj)
        data = pd.DataFrame(data)
        # Take only samples with certain category
        self.data = data.loc[data['density_bin'] == self.category, :]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return dict(self.data.iloc[idx, :])

In [31]:
test_dset = NewsroomDataset(test_path, "extractive")
testloader = torch.utils.data.DataLoader(test_dset, batch_size=batch_size)   

### Verify that the dataloader works like we want

In [38]:
tset = next(iter(testloader))
text, summary = itemgetter('text', 'summary')(tset)
print(f"Text-to-summarize: {text}, \n\nsummarization: {summary}")

Text-to-summarize: ['BANGALORE, India, June 4 \x97 The world\'s biggest computer services company could not have chosen a more appropriate setting to lay out its strategy for staying on top.\n\nA building housing I.B.M.\'s software laboratory and application service teams on the company\'s corporate campus in Bangalore, India.\n\nOn Tuesday, on the expansive grounds of the Bangalore Palace, a colonial-era mansion once inhabited by a maharajah, the chairman and chief executive of I.B.M., Samuel J. Palmisano, will address 10,000 Indian employees. He will share the stage with A. P. J. Abdul Kalam, India\'s president, and Sunil Mittal, chairman of the country\'s largest cellular services provider, Bharti Tele-Ventures. An additional 6,500 employees will look in on the town hall-style meeting by satellite from other Indian cities.\n\nOn the same day, Mr. Palmisano and other top executives will meet here with investment analysts and local customers to showcase I.B.M.\'s global integration ca

## 2. Initialize model and do predictions

## 3. Performance evaluation and results